In [1]:
## Neural nets in pytorch
 # defined in torch.nn
 # relies on autograd to define models and differentiate them
 # nn.Module contains layers and a forward(input) method that returns the net's output

In [2]:
## Typical training procedure (we know most of this already, but always good to review)
 # Define the net that has some weights (learnable parameters)
 # Process some inputs through the network
 # Compute the loss for those inputs
 # Propogate gradients back to the paramters
 # Update the weights of the network using your sample rule (e.g. new_weight = old_weight - learning_rate * gradient)

In [6]:
## We're going to build the classic digit image classifier :)
import torch
import torch.nn as nn
import torch.nn.functional as F

# Note: this is all but copy and pasted from the tutorial for the sake of time
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution kernel
        # TODO: !DF! Quick review on kernel sizing here: https://stats.stackexchange.com/questions/296679/what-does-kernel-size-mean/296701
        self.conv1 = nn.Conv2d(1,6,3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # Output * Conv2 input * Conv1 output
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)  # 10 output features representing the digit being 0...9
    
    def forward(self, x):
        # Max pooling over a (2,2) window
        # TODO: !DF! Review on ReLu here: https://machinelearningmastery.com/rectified-linear-activation-function-for-deep-learning-neural-networks/
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:] # All features except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features        

In [27]:
net = Net()
params = list(net.parameters())
print(len(params))
print(params[0].size())  # Conv1's .weight

10
torch.Size([6, 1, 3, 3])


In [34]:
# Let's try a random input of 32x32
input = torch.randn(1, 32, 32).unsqueeze(0)
out = net.forward(input)
out

tensor([[ 0.1279, -0.0395,  0.0909, -0.0368,  0.0572,  0.0068,  0.0393,  0.0672,
         -0.0743, -0.1268]], grad_fn=<AddmmBackward>)

In [22]:
576 / 16

36.0